In [1]:
import requests
import csv
import pandas

api_key = "4JJ6E759L3IN858J"
url = "https://www.alphavantage.co/query"


ModuleNotFoundError: No module named 'requests'

In [12]:
def getListingStatus():
    params = {
        "function"  :   "LISTING_STATUS",
        "apikey"    :   api_key
    }
    response = requests.request("GET",url,params=params)
    return response
getListingStatus()

<Response [200]>

In [13]:
def test_statusCode(response):
    status_code = response.status_code
    assert status_code == 200

listingStatus = getListingStatus()

test_statusCode(listingStatus)

In [14]:
def contentToCSV(response,filename):
    decoded_content = response.content.decode('utf-8')
    content = csv.reader(decoded_content.splitlines(), delimiter=',')
    content_list = list(content)
    with open(filename,'w') as m:
        for row in content_list:
            m.write(','.join(row))
            m.write('\n')

# listingStatus = getListingStatus()
# contentToCSV(listingStatus,'listing_status.csv')

In [15]:
def test_fileOut(filename):
    out = open(filename,'r')
    out.close()
    assert out != FileNotFoundError

file = 'listing_status.csv'
test_fileOut(file)

In [16]:
def listingDataFrame():
    with open('listing_status.csv','r') as data:
        listing_df = pandas.read_csv(data)
    return listing_df
listingDataFrame()

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaN,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaN,Active
2,AAA,AXS FIRST PRIORITY CLO BOND ETF,NYSE ARCA,ETF,2020-09-09,NaN,Active
3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,NaN,Active
4,AAC,Ares Acquisition Corporation - Class A,NYSE,Stock,2021-03-25,NaN,Active
...,...,...,...,...,...,...,...
11813,ZYNE,Zynerba Pharmaceuticals Inc,NASDAQ,Stock,2015-08-05,NaN,Active
11814,ZYRX,Global Earnings Capital Ltd,NASDAQ,Stock,2007-07-13,NaN,Active
11815,ZYXI,Zynex Inc,NASDAQ,Stock,2012-07-23,NaN,Active
11816,ZZK,NaN,NYSE ARCA,Stock,2020-07-22,NaN,Active


In [17]:
def test_entryCount():
    count_csv = 0
    with open('listing_status.csv','r') as m:
        for i in m:
            count_csv += 1
        count_csv -= 1
    count_df = listingDataFrame().shape[0]
    if count_csv == count_df:
        return True
    else:
        return False
test_entryCount()

True

In [18]:
def getFullDailyData(symbol):
    params = {
        "function"  :   "TIME_SERIES_DAILY_ADJUSTED",
        "symbol"    :   symbol,
        "apikey"    :   api_key,
        "datatype"  :   "csv",
        "outputsize":   "full"
    }
    response = requests.request("GET",url,params=params)
    return response
getFullDailyData('ITCM')



<Response [200]>

In [19]:
def test_checkContent(response):
    decoded_content = response.content.decode('utf-8')
    err = "Error" in decoded_content
    assert err == False
    
test_checkContent(getFullDailyData('IBM'))
    

In [20]:
ibm_raw = getFullDailyData("IBM")
contentToCSV(ibm_raw,'IBM_full.csv')

def makeDataFrame(flename):
    df = pandas.read_csv('IBM_full.csv')
    return df

ibm_df = makeDataFrame(ibm_raw)

In [21]:
ibm_df['symbol'] = 'IBM'

In [22]:
ibm_df['timestamp'] = pandas.to_datetime(ibm_df['timestamp'])
ibm_df


,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,symbol
0,2023-01-06,142.38,144.250,141.5800,143.70,143.700000,3574042,0.0,1.0,IBM
1,2023-01-05,142.44,142.498,140.0100,141.11,141.110000,2866648,0.0,1.0,IBM
2,2023-01-04,142.07,143.615,141.3675,142.60,142.600000,3869236,0.0,1.0,IBM
3,2023-01-03,141.10,141.900,140.4800,141.55,141.550000,3338829,0.0,1.0,IBM
4,2022-12-30,140.54,140.900,139.4500,140.89,140.890000,2858110,0.0,1.0,IBM
...,...,...,...,...,...,...,...,...,...,...
5829,1999-11-05,92.75,92.940,90.1900,90.25,49.728782,13737600,0.0,1.0,IBM
5830,1999-11-04,94.44,94.440,90.0000,91.56,50.450607,16697600,0.0,1.0,IBM
5831,1999-11-03,95.87,95.940,93.5000,94.37,51.998949,10369100,0.0,1.0,IBM
5832,1999-11-02,96.75,96.810,93.6900,94.81,52.241394,11105400,0.0,1.0,IBM


In [23]:
type(ibm_df['timestamp'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [24]:
def makeDataFrame(filename):
    df = pandas.read_csv(filename)
    return df

ibm_df = makeDataFrame('IBM_full.csv')

In [25]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=ibm_df['timestamp'],
                open=ibm_df['open'],
                high=ibm_df['high'],
                low=ibm_df['low'],
                close=ibm_df['close'])])

fig.show()   

In [29]:
def getFullMonthlyData(symbol):
    params = {
        "function"  :   "TIME_SERIES_MONTHLY_ADJUSTED",
        "symbol"    :   symbol,
        "apikey"    :   api_key,
        "datatype"  :   "csv",
        "outputsize":   "full"
    }
    response = requests.request("GET",url,params=params)
    return response


In [30]:

ibm_raw = getFullMonthlyData('IBM')
contentToCSV(ibm_raw,'IBM_monthly.csv')

ibm_df = makeDataFrame('IBM_monthly.csv')

In [31]:
def plotCandlesticks():
    fig = go.Figure(data=[go.Candlestick(x=ibm_df['timestamp'],
                open=ibm_df['open'],
                high=ibm_df['high'],
                low=ibm_df['low'],
                close=ibm_df['close'])])

    

    fig.show()  
    
plotCandlesticks()

In [32]:
import sqlite3

In [49]:
conn = sqlite3.connect("stocks_data.sqlite")

In [50]:
df = pandas.read_csv("IBM_full.csv")
df['symbol'] = 'IBM'
df.to_sql("HistoricalData", conn, if_exists='replace', index=False)

5834

In [36]:
cur = conn.cursor()

In [51]:
listing = pandas.read_csv("listing_status.csv")
listing.to_sql("Listing",conn,if_exists='append', index=False)

11818

In [52]:
df

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,symbol
0,2023-01-06,142.38,144.250,141.5800,143.70,143.700000,3574042,0.0,1.0,IBM
1,2023-01-05,142.44,142.498,140.0100,141.11,141.110000,2866648,0.0,1.0,IBM
2,2023-01-04,142.07,143.615,141.3675,142.60,142.600000,3869236,0.0,1.0,IBM
3,2023-01-03,141.10,141.900,140.4800,141.55,141.550000,3338829,0.0,1.0,IBM
4,2022-12-30,140.54,140.900,139.4500,140.89,140.890000,2858110,0.0,1.0,IBM
...,...,...,...,...,...,...,...,...,...,...
5829,1999-11-05,92.75,92.940,90.1900,90.25,49.728782,13737600,0.0,1.0,IBM
5830,1999-11-04,94.44,94.440,90.0000,91.56,50.450607,16697600,0.0,1.0,IBM
5831,1999-11-03,95.87,95.940,93.5000,94.37,51.998949,10369100,0.0,1.0,IBM
5832,1999-11-02,96.75,96.810,93.6900,94.81,52.241394,11105400,0.0,1.0,IBM


In [55]:
apple_raw = getFullDailyData('AAPL')

def decodeContent(response):
    decoded_content = response.content.decode('utf-8')
    content = csv.reader(decoded_content.splitlines(), delimiter=',')
    content_list = list(content)
    return content_list


[['timestamp',
  'open',
  'high',
  'low',
  'close',
  'adjusted_close',
  'volume',
  'dividend_amount',
  'split_coefficient'],
 ['2023-01-06',
  '126.01',
  '130.29',
  '124.89',
  '129.62',
  '129.62',
  '87754715',
  '0.0000',
  '1.0'],
 ['2023-01-05',
  '127.13',
  '127.77',
  '124.76',
  '125.02',
  '125.02',
  '80962708',
  '0.0000',
  '1.0'],
 ['2023-01-04',
  '126.89',
  '128.6557',
  '125.08',
  '126.36',
  '126.36',
  '89113633',
  '0.0000',
  '1.0'],
 ['2023-01-03',
  '130.28',
  '130.9',
  '124.17',
  '125.07',
  '125.07',
  '112117471',
  '0.0000',
  '1.0'],
 ['2022-12-30',
  '128.41',
  '129.95',
  '127.43',
  '129.93',
  '129.93',
  '77034209',
  '0.0000',
  '1.0'],
 ['2022-12-29',
  '127.99',
  '130.4814',
  '127.73',
  '129.61',
  '129.61',
  '75703710',
  '0.0000',
  '1.0'],
 ['2022-12-28',
  '129.67',
  '131.0275',
  '125.87',
  '126.04',
  '126.04',
  '85438391',
  '0.0000',
  '1.0'],
 ['2022-12-27',
  '131.38',
  '131.41',
  '128.72',
  '130.03',
  '130.03',
  

In [61]:
apple_csv = decodeContent(apple_raw)
apple_df = pandas.DataFrame(apple_csv[1:],columns=apple_csv[0])

In [64]:
apple_df['symbol'] = 'AAPL'
apple_df.to_sql("HistoricalData",conn,if_exists='append',index=False)

5834

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2023-01-06,126.01,130.29,124.89,129.62,129.62,87754715,0.0000,1.0
1,2023-01-05,127.13,127.77,124.76,125.02,125.02,80962708,0.0000,1.0
2,2023-01-04,126.89,128.6557,125.08,126.36,126.36,89113633,0.0000,1.0
3,2023-01-03,130.28,130.9,124.17,125.07,125.07,112117471,0.0000,1.0
4,2022-12-30,128.41,129.95,127.43,129.93,129.93,77034209,0.0000,1.0
...,...,...,...,...,...,...,...,...,...
5829,1999-11-05,84.62,88.37,84.0,88.31,0.671185647632425,3721500,0.0000,1.0
5830,1999-11-04,82.06,85.37,80.62,83.62,0.635540073095045,3384700,0.0000,1.0
5831,1999-11-03,81.62,83.25,81.0,81.5,0.619427361363862,2932700,0.0000,1.0
5832,1999-11-02,78.0,81.69,77.31,80.25,0.609926941711042,3564600,0.0000,1.0
